In [ ]:
## Tabulate

In [ ]:
import pandas as pd

In [ ]:
from tabulate import tabulate

In [ ]:
# Example data: replace this with your actual model metrics

In [ ]:
data = [

In [ ]:
    {"Model": "Logistic Regression", "Type": "Machine Learning", "Accuracy": 0.74, "Precision": 0.74, "Recall": 0.74, "F1-Score": 0.71, "ROC AUC Score": 0.85},

In [ ]:
    {"Model": "SVM", "Type": "Machine Learning", "Accuracy": 0.76, "Precision": 0.75, "Recall": 0.76, "F1-Score": 0.75, "ROC AUC Score": 0.87},

In [ ]:
    {"Model": "Random Forest", "Type": "Machine Learning", "Accuracy": 0.76, "Precision": 0.75, "Recall": 0.76, "F1-Score": 0.73, "ROC AUC Score": 0.83},

In [ ]:
    {"Model": "CNN", "Type": "Deep Learning", "Accuracy": 0.76, "Precision": 0.75, "Recall": 0.76, "F1-Score": 0.75, "ROC AUC Score": 0.85},

In [ ]:
    {"Model": "LSTM", "Type": "Deep Learning", "Accuracy": 0.66, "Precision": 0.68, "Recall": 0.66, "F1-Score": 0.67, "ROC AUC Score": 0.80},

In [ ]:
    {"Model": "BiLSTM", "Type": "Deep Learning", "Accuracy": 0.63, "Precision": 0.64, "Recall": 0.63, "F1-Score": 0.64, "ROC AUC Score": 0.77},

In [ ]:
    {"Model": "BERT", "Type": "Transformer Model", "Accuracy": 0.85, "Precision": 0.86, "Recall": 0.85, "F1-Score": 0.85, "ROC AUC Score": 0.94},

In [ ]:
    {"Model": "RoBERTa", "Type": "Transformer Model", "Accuracy": 0.87, "Precision": 0.87, "Recall": 0.87, "F1-Score": 0.86, "ROC AUC Score": 0.95},

In [ ]:
]

In [ ]:
# Creating a DataFrame

In [ ]:
df = pd.DataFrame(data)